## CONFIG

In [1]:
import pdfplumber
import re
import json
import os
import html
import PyPDF2
import pandas as pd
from bs4 import BeautifulSoup



from collections import Counter

from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

In [2]:
codice_galattico = "../Hackapizza Dataset/Codice Galattico/Codice Galattico.pdf"
manuale_cucina = "../Hackapizza Dataset/Misc/Manuale di Cucina.pdf"

domande = "../Hackapizza Dataset/domande.csv"
piatti = "../Hackapizza Dataset/Misc/dish_mapping.json"
pianeti = "../Hackapizza Dataset/Misc/Distanze.csv"

blog_1 = "../Hackapizza Dataset/Blogpost/blog_etere_del_gusto.html"
blog_2 = "../Hackapizza Dataset/Blogpost/blog_sapore_del_dune.html"


load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [3]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

In [4]:
def estrai_testo_pdf(percorso_pdf):
    with open(percorso_pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        testo = ""
        for pagina in reader.pages:
            testo += pagina.extract_text()
    return testo

def estrai_testo_html(percorso_html):
    """Estrae il testo da un file HTML."""
    with open(percorso_html, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
        return soup.get_text(separator=" ", strip=True)

## METODI

In [ ]:
def estrai_valorizzazioni(testo, categoria, llm, driver):
    """
    Estrae le valorizzazioni di entità di una specifica categoria dal testo,
    assicurando che rispettino lo schema del database.
    """
    # Definizione delle proprietà ammesse per ogni tipo di entità
    proprieta_per_categoria = {
        "Tecnica": ["nome", "tipo", "caratteristica", "uso", "applicazione", "effetto", 
                   "descrizione", "Svantaggi", "Vantaggi", "Tipo", "comeFunziona"],
        "Ristorante": ["nome", "Chef", "Location", "description", "cucina", "voto"],
        "Piatto": ["nome", "numero"],
        "Ingrediente": ["nome", "tipo", "origine", "sapore", "Aspetto", "Proprietà"],
        "Licenza": ["nome", "Livello", "Descrizione"],
        "Pianeta": ["nome", "ambiente", "clima", "intolleranzaLattosio", "intolleranza", 
                    "descrizione", "caratteristiche", "biomi", "magia"]
    }
    
    # Verifica che la categoria sia valida
    if categoria not in proprieta_per_categoria:
        print(f"Errore: la categoria '{categoria}' non è valida.")
        return []
    
    # Ottieni le proprietà valide per questa categoria
    proprieta_valide = proprieta_per_categoria[categoria]
    proprieta_str = ", ".join(proprieta_valide)
    
    # Genera un prompt più preciso in base allo schema
    prompt = f"""
    Contesto: questo testo rientra in una collezione di documenti che parlano di Ristoranti nello spazio (su Pianeti) e sui loro Menu, Piatti, Ingredienti, Tecniche e Licenze.
    
    Il testo seguente potrebbe contenere informazioni su entità di tipo '{categoria}'.
    
    In output voglio tutte le possibili istanze di '{categoria}' come entità da caricare su neo4j.
    
    Per ogni '{categoria}' recupera SOLO ed ESCLUSIVAMENTE queste proprietà (se presenti nel testo):
    {proprieta_str}
    
    NOTE IMPORTANTI:
    1. Non inventare proprietà che non sono nell'elenco fornito
    2. Non modificare i nomi delle proprietà
    3. Per la proprietà 'nome' usa SEMPRE una stringa singola, NON una lista
    4. Se "{categoria}" ha "Livello" o "numero", questi devono essere valori numerici interi
    
    Restituisci SOLO il JSON valido e completo, senza formattazione markdown o decoratori.
    Il formato deve essere:
    [
      {{"nome": "nome_entità", "proprietà2": "valore2", ...}}
    ]
    
    Testo:
    {testo}
    """
    
    # Chiamata al modello LLM per ottenere la valorizzazione delle entità
    response = llm.invoke(prompt)
    content = response.content.strip()
    
    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('```json'):
        content = content[7:].strip()
    elif content.startswith('json'):
        content = content[4:].strip()
    
    # Tentativo di riparare il JSON
    try:
        valorizzazioni = json.loads(content)
    except json.JSONDecodeError as e:
        print(f"Errore nel parsing JSON per entità {categoria}: {str(e)}")
        try:
            valorizzazioni = json.loads(repair_json(content))
        except Exception:
            # Fallback con richiesta all'LLM
            prompt_repair = f"""
            Correggi questo JSON per Neo4j e assicurati che sia valido e completo.
            Ogni entità deve avere almeno la proprietà 'nome'.
            Le proprietà valide per '{categoria}' sono: {proprieta_str}.
            Assicurati che 'nome' sia sempre una stringa singola e non una lista.
            
            JSON da correggere:
            {content}
            """
            repaired = llm.invoke(prompt_repair).content.strip()
            
            # Pulizia del JSON riparato
            if repaired.startswith('```') and repaired.endswith('```'):
                repaired = '\n'.join(repaired.split('\n')[1:-1])
            if repaired.startswith('```json'):
                repaired = repaired[7:].strip()
            elif repaired.startswith('json'):
                repaired = repaired[4:].strip()
                
            try:
                valorizzazioni = json.loads(repaired)
            except:
                print(f"Impossibile riparare il JSON per le entità di tipo {categoria}")
                valorizzazioni = []
    
    # Assicurati che il risultato sia una lista
    if not isinstance(valorizzazioni, list):
        valorizzazioni = [valorizzazioni]
    
    # Validazione e pulizia delle entità estratte
    entita_valide = []
    for entita in valorizzazioni:
        # Verifica che ci sia la proprietà 'nome'
        if 'nome' not in entita:
            continue
        
        # Verifica che nome sia una stringa e non una lista
        if isinstance(entita['nome'], list):
            entita['nome'] = entita['nome'][0] if entita['nome'] else ""
        
        # Rimuovi proprietà non valide
        entita_pulita = {'nome': entita['nome']}
        for prop in proprieta_valide:
            if prop in entita and prop != 'nome':
                # Converti i valori numerici se necessario
                if prop in ['Livello', 'numero'] and categoria in ['Licenza', 'Piatto']:
                    try:
                        entita_pulita[prop] = int(entita[prop])
                    except (ValueError, TypeError):
                        if isinstance(entita[prop], str) and entita[prop].strip():
                            entita_pulita[prop] = 1  # Default value
                else:
                    entita_pulita[prop] = entita[prop]
        
        entita_valide.append(entita_pulita)
    
    return entita_valide


def estrai_valorizzazioni_relazioni(testo, nome_relazione, tipo_entita1, tipo_entita2, entita1, entita2, llm, driver):
    """
    Estrae le relazioni specifiche tra entità di due tipi, rispettando lo schema del database.
    """
    # Definizione delle relazioni valide e loro proprietà
    relazioni_valide = {
        "OFFERTA": {"origine": "Ristorante", "destinazione": "Piatto", "proprieta": []},
        "CONTIENE": {"origine": "Piatto", "destinazione": "Ingrediente", "proprieta": []},
        "PREPARATO": {"origine": "Piatto", "destinazione": "Tecnica", "proprieta": []},
        "SITUATO": {"origine": "Ristorante", "destinazione": "Pianeta", "proprieta": ["descrizione"]},
        "HA": {"origine": "Ristorante", "destinazione": "Licenza", "proprieta": ["Livello"]},
        "DISTANZA": {"origine": "Pianeta", "destinazione": "Pianeta", "proprieta": ["km"]}
    }
    
    # Verifica che la relazione sia valida
    if nome_relazione not in relazioni_valide:
        print(f"Errore: la relazione '{nome_relazione}' non è valida.")
        return []
    
    # Verifica che i tipi di entità corrispondano alla relazione
    rel_info = relazioni_valide[nome_relazione]
    if rel_info["origine"] != tipo_entita1 or rel_info["destinazione"] != tipo_entita2:
        print(f"Errore: la relazione '{nome_relazione}' deve essere tra '{rel_info['origine']}' e '{rel_info['destinazione']}'.")
        return []
    
    # Ottieni le proprietà valide per questa relazione
    proprieta_valide = rel_info["proprieta"]
    proprieta_str = ", ".join(proprieta_valide) if proprieta_valide else "nessuna"
    
    # Preparazione delle entità per il prompt
    entita1_nomi = [e.get("nome", "") for e in (entita1 if isinstance(entita1, list) else [entita1]) if e.get("nome")]
    entita2_nomi = [e.get("nome", "") for e in (entita2 if isinstance(entita2, list) else [entita2]) if e.get("nome")]
    
    # Stringa con le proprietà e i loro tipi
    proprieta_info = ""
    if "km" in proprieta_valide:
        proprieta_info += "\n- 'km' deve essere un numero intero"
    if "Livello" in proprieta_valide:
        proprieta_info += "\n- 'Livello' deve essere un numero intero"
    if "descrizione" in proprieta_valide:
        proprieta_info += "\n- 'descrizione' deve essere una stringa"
    
    # Genera un prompt per estrarre le relazioni
    prompt = f"""
    Contesto: questo testo rientra in una collezione di documenti che parlano di Ristoranti nello spazio (su Pianeti) e sui loro Menu, Piatti, Ingredienti, Tecniche e Licenze.
    
    Il testo seguente potrebbe contenere informazioni su relazioni di tipo '{nome_relazione}' tra entità di tipo '{tipo_entita1}' e '{tipo_entita2}'.
    
    In output voglio tutte le relazioni di tipo '{nome_relazione}' come triple da caricare su neo4j.
    
    Per ogni relazione recupera:
    - origine: nome dell'entità di tipo '{tipo_entita1}'
    - destinazione: nome dell'entità di tipo '{tipo_entita2}'
    - proprietà: {proprieta_str} {proprieta_info}
    
    Ecco le entità di tipo '{tipo_entita1}' già identificate:
    {entita1_nomi}
    
    Ecco le entità di tipo '{tipo_entita2}' già identificate:
    {entita2_nomi}
    
    NOTE IMPORTANTI:
    1. Identifica SOLO relazioni tra entità presenti negli elenchi forniti
    2. Se non ci sono relazioni evidenti, restituisci un array vuoto
    
    Restituisci SOLO il JSON valido e completo, senza formattazione markdown o decoratori.
    Il formato deve essere:
    [
      {{"origine": "nome_entita1", "destinazione": "nome_entita2", "proprietà": {{{proprieta_str}}}}},
    ]
    
    Testo:
    {testo}
    """
    
    # Chiamata al modello LLM per ottenere la valorizzazione delle relazioni
    response = llm.invoke(prompt)
    content = response.content.strip()
    
    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('```json'):
        content = content[7:].strip()
    elif content.startswith('json'):
        content = content[4:].strip()
    
    # Tentativo di riparare il JSON
    try:
        relazioni = json.loads(content)
    except json.JSONDecodeError as e:
        print(f"Errore nel parsing JSON delle relazioni {nome_relazione}: {str(e)}")
        try:
            relazioni = json.loads(repair_json(content))
        except Exception:
            # Fallback con richiesta all'LLM
            prompt_repair = f"""
            Correggi questo JSON per Neo4j e assicurati che sia valido e completo.
            Ogni relazione deve avere 'origine', 'destinazione', e le seguenti proprietà se richieste: {proprieta_str}.
            
            JSON da correggere:
            {content}
            """
            repaired = llm.invoke(prompt_repair).content.strip()
            
            # Pulizia del JSON riparato
            if repaired.startswith('```') and repaired.endswith('```'):
                repaired = '\n'.join(repaired.split('\n')[1:-1])
            if repaired.startswith('```json'):
                repaired = repaired[7:].strip()
            elif repaired.startswith('json'):
                repaired = repaired[4:].strip()
                
            try:
                relazioni = json.loads(repaired)
            except:
                print(f"Impossibile riparare il JSON delle relazioni {nome_relazione}")
                relazioni = []
    
    # Assicurati che il risultato sia una lista
    if not isinstance(relazioni, list):
        relazioni = [relazioni]
    
    # Validazione e pulizia delle relazioni estratte
    relazioni_valide_lista = []
    for rel in relazioni:
        # Verifica che ci siano origine e destinazione
        if 'origine' not in rel or 'destinazione' not in rel:
            continue
        
        # Verifica che origine e destinazione siano nelle liste fornite
        if rel['origine'] not in entita1_nomi or rel['destinazione'] not in entita2_nomi:
            continue
        
        # Crea la relazione pulita
        rel_pulita = {
            'origine': rel['origine'],
            'destinazione': rel['destinazione']
        }
        
        # Aggiungi solo le proprietà valide
        if proprieta_valide:
            proprieta = {}
            for prop in proprieta_valide:
                if prop in rel.get('proprietà', {}):
                    if prop == 'km' or prop == 'Livello':
                        try:
                            proprieta[prop] = int(rel['proprietà'][prop])
                        except (ValueError, TypeError):
                            proprieta[prop] = 0  # Default value
                    else:
                        proprieta[prop] = rel['proprietà'][prop]
            
            rel_pulita['proprietà'] = proprieta
        
        relazioni_valide_lista.append(rel_pulita)
    
    return relazioni_valide_lista

In [6]:
import json
from json_repair import repair_json

def estrai_relazioni(testo, relazioni_info, llm, driver):
    """
    Estrae entità e relazioni tra di esse da un testo.
    
    Args:
        testo: Il testo da cui estrarre le relazioni
        relazioni_info: Dizionario nella forma {'NOME_RELAZIONE': ['Tipo_Entità1', 'Tipo_Entità2']}
        llm: Modello di linguaggio da utilizzare
        driver: Driver Neo4j
        
    Returns:
        Dizionario contenente entità e relazioni estratte
    """
    risultati = {}
    entita_tipi = set()
    
    # Raccogli tutti i tipi di entità unici dalle relazioni
    for _, [tipo_entita1, tipo_entita2] in relazioni_info.items():
        entita_tipi.add(tipo_entita1)
        entita_tipi.add(tipo_entita2)
    
    # Estrai tutte le entità una sola volta per tipo
    for tipo_entita in entita_tipi:
        print(f"Estraendo entità di tipo '{tipo_entita}'...")
        entita = estrai_valorizzazioni(testo, tipo_entita, llm, driver)
        risultati[tipo_entita] = entita
    
    # Ora estrai le relazioni
    for nome_relazione, [tipo_entita1, tipo_entita2] in relazioni_info.items():
        print(f"Estraendo relazioni '{nome_relazione}' tra '{tipo_entita1}' e '{tipo_entita2}'...")
        
        # Estrai le relazioni tra entità1 e entità2
        relazioni = estrai_valorizzazioni_relazioni(
            testo, 
            nome_relazione, 
            tipo_entita1, 
            tipo_entita2, 
            risultati[tipo_entita1], 
            risultati[tipo_entita2], 
            llm, 
            driver
        )
        
        # Salva le relazioni nel risultato
        risultati[nome_relazione] = relazioni
        
    return risultati

In [24]:
def carica_entita_e_relazioni_su_neo4j(risultati, relazioni_info, driver):
    """
    Carica entità e relazioni su Neo4j gestendo strutture dati nidificate
    
    Args:
        risultati: Dizionario contenente entità e relazioni
        relazioni_info: Dizionario delle relazioni come riferimento
        driver: Driver Neo4j
    """
    # Crea un set di tutti i tipi di relazione
    tipi_relazione = set(relazioni_info.keys())
    
    # Contatori per verifica
    entita_create = 0
    relazioni_create = 0
    
    # Verifica preliminare che il driver sia connesso
    try:
        with driver.session() as session:
            result = session.run("RETURN 1 AS test")
            test_value = result.single()["test"]
            if test_value != 1:
                print("ERRORE: Connessione al database non funzionante")
                return
            print("Connessione al database verificata con successo")
    except Exception as e:
        print(f"ERRORE: Impossibile connettersi al database: {str(e)}")
        return
    
    # Carica le entità
    for tipo_entita, entita_data in risultati.items():
        # Salta le chiavi che sono nomi di relazione
        if tipo_entita in tipi_relazione:
            continue
        
        print(f"Elaborazione per tipo: {tipo_entita}")
        print(f"Tipo di dato: {type(entita_data).__name__}")
        
        # Gestione speciale per struttura dati nidificata
        if isinstance(entita_data, dict) and tipo_entita in entita_data:
            print(f"Rilevata struttura nidificata per {tipo_entita}")
            if isinstance(entita_data[tipo_entita], list):
                entita_list = entita_data[tipo_entita]
            else:
                entita_list = [entita_data[tipo_entita]]
            print(f"Estratti {len(entita_list)} elementi dalla struttura nidificata")
        elif isinstance(entita_data, list):
            entita_list = entita_data
        elif isinstance(entita_data, dict) and "NOME" in entita_data:  # caso di singolo oggetto
            entita_list = [entita_data]
        else:
            print(f"ERRORE: Formato non supportato per {tipo_entita}: {entita_data}")
            continue
        
        print(f"Caricando {len(entita_list)} entità di tipo {tipo_entita}...")
        
        for entita in entita_list:
            print(f"Debug: Entità esaminata: {entita}")  # Aggiungi un log per vedere l'intera entità
            
            # Verifica l'accesso alla chiave "nome"
            if "nome" in entita:
                print(f"DEBUG: Trovata chiave 'nome' con valore: {entita['nome']}")
            else:
                print(f"ERRORE: Chiave 'nome' non trovata nella entità: {entita}")
            
            nome = entita.get("nome", "").strip()  # Accedi alla chiave "nome" in modo case-insensitive
            
            # Debug: Mostra se il nome è stato trovato
            if not nome:
                print(f"ERRORE: Entità di tipo {tipo_entita} senza nome: {entita}")
                continue
            
            parametri = entita.get("PARAMETRI", {})
            
            print(f"Creazione entità: {nome} (tipo: {tipo_entita})")
            
            # Usa una transazione esplicita
            with driver.session() as session:
                tx = session.begin_transaction()
                try:
                    # Crea la query base
                    query = f"MERGE (e:{tipo_entita} {{nome: $nome}})"
                    
                    # Se ci sono parametri, aggiungi SET
                    if parametri:
                        query += " SET "
                        set_clauses = []
                        for k in parametri.keys():
                            # Gestione speciale per parametri che sono dizionari
                            if isinstance(parametri[k], dict):
                                # Converti sotto-dizionari in formato JSON o string
                                set_clauses.append(f"e.{k} = $param_{k}")
                            else:
                                set_clauses.append(f"e.{k} = ${k}")
                        query += ", ".join(set_clauses)
                    
                    query += " RETURN e"  # Ritorna il nodo per conferma
                    
                    # Prepara i parametri per la query
                    params = {"nome": nome}
                    for k, v in parametri.items():
                        if isinstance(v, dict):
                            params[f"param_{k}"] = v  # Neo4j supporta JSON per le proprietà
                        else:
                            params[k] = v
                    
                    print(f"Esecuzione query: {query}")
                    result = tx.run(query, **params)
                    record = result.single()
                    if record:
                        node = record.get("e")
                        print(f"Entità creata e confermata: {nome} di tipo {tipo_entita}")
                        entita_create += 1
                    else:
                        print(f"ATTENZIONE: Entità {nome} non confermata dopo l'esecuzione della query")
                    
                    # Esplicita conferma della transazione
                    tx.commit()
                except Exception as e:
                    tx.rollback()
                    print(f"ERRORE nel caricamento dell'entità {nome}: {str(e)}")
                    print(f"Query: {query}")
                    print(f"Parametri: {params}")
    
    # Resto del codice per caricare le relazioni...


In [18]:
import re

def chunk_text(text, max_chars=3000, overlap=300):
    """
    Divide un testo in chunk di massimo `max_chars` caratteri, con `overlap` tra i chunk.
    
    Args:
        text (str): Il testo da spezzare in chunk.
        max_chars (int): Numero massimo di caratteri per chunk.
        overlap (int): Numero di caratteri di sovrapposizione tra i chunk.
    
    Returns:
        list: Lista di chunk di testo.
    """
    # Pulizia del testo per evitare problemi di spazi extra
    text = re.sub(r'\n+', '\n', text.strip())

    chunks = []
    start = 0
    while start < len(text):
        end = start + max_chars
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap  # Sposta l'inizio del prossimo chunk per avere overlap
    
    return chunks

## MAIN

In [9]:
# Carica il file CSV
df_domande = pd.read_csv(domande)
testo_domande = df_domande.to_string()  # Converti il CSV in testo

# Carica il file CSV
df_pianeti = pd.read_csv(pianeti)
testo_pianeti = df_pianeti.to_string()  # Converti il CSV in testo

# Esegui l'analisi del PDF
testo_manuale = estrai_testo_pdf(manuale_cucina)
# Esegui l'analisi del PDF
testo_codice = estrai_testo_pdf(codice_galattico)
# Esempio di utilizzo:
testo_blog1 = estrai_testo_html(blog_1)
# Esempio di utilizzo:
testo_blog2 = estrai_testo_html(blog_2)

In [10]:
relazioni = {'CONTIENE': ['Piatto', 'Ingrediente'], 'PREPARATO': ['Piatto', 'Tecnica'], 'OFFERTA': ['Ristorante', 'Piatto'], 'SITUATO': ['Ristorante', 'Pianeta'], 'HA': ['Ristorante', 'Licenza']}

In [23]:
# Estrai entità e relazioni
chunks = chunk_text(testo_manuale, 3000, 400)
for chunk in chunks:
    risultati = estrai_relazioni(chunk, relazioni, llm, driver)
    carica_entita_e_relazioni_su_neo4j(risultati, relazioni, driver)

Estraendo entità di tipo 'Pianeta'...
Estraendo entità di tipo 'Licenza'...
Estraendo entità di tipo 'Piatto'...
Estraendo entità di tipo 'Tecnica'...
Estraendo entità di tipo 'Ristorante'...
Estraendo entità di tipo 'Ingrediente'...
Estraendo relazioni 'CONTIENE' tra 'Piatto' e 'Ingrediente'...
Estraendo relazioni 'PREPARATO' tra 'Piatto' e 'Tecnica'...
Estraendo relazioni 'OFFERTA' tra 'Ristorante' e 'Piatto'...
Estraendo relazioni 'SITUATO' tra 'Ristorante' e 'Pianeta'...
Estraendo relazioni 'HA' tra 'Ristorante' e 'Licenza'...
Connessione al database verificata con successo
Elaborazione per tipo: Pianeta
Tipo di dato: list
Caricando 0 entità di tipo Pianeta...
Elaborazione per tipo: Licenza
Tipo di dato: list
Caricando 14 entità di tipo Licenza...
Debug: Entità esaminata: {'nome': 'Psionica', 'Livello': 0, 'Descrizione': 'Posseduta da tutti se non diversamente specificato. Tipica degli esseri senzienti.'}
ERRORE: Entità di tipo Licenza senza nome: {'nome': 'Psionica', 'Livello': 0,

In [137]:
# Estrai entità e relazioni
chunks = chunk_text(testo_codice, 3000, 400)
for chunk in chunks:
    risultati = estrai_relazioni(chunk, relazioni, llm, driver)
    carica_entita_e_relazioni_su_neo4j(risultati, relazioni, driver)

Estraendo entità di tipo 'Piatto'...
Estraendo entità di tipo 'Licenza'...
Estraendo entità di tipo 'Pianeta'...
Estraendo entità di tipo 'Ingrediente'...
Estraendo entità di tipo 'Tecnica'...
Estraendo entità di tipo 'Ristorante'...
Estraendo relazioni 'CONTIENE' tra 'Piatto' e 'Ingrediente'...
Estraendo relazioni 'PREPARATO' tra 'Piatto' e 'Tecnica'...
Estraendo relazioni 'OFFERTA' tra 'Ristorante' e 'Piatto'...
Estraendo relazioni 'SITUATO' tra 'Ristorante' e 'Pianeta'...
Estraendo relazioni 'HA' tra 'Ristorante' e 'Licenza'...
Connessione al database verificata con successo
Elaborazione per tipo: Piatto
Tipo di dato: list
Caricando 0 entità di tipo Piatto...
Elaborazione per tipo: Licenza
Tipo di dato: list
Caricando 2 entità di tipo Licenza...
Creazione entità: Decreto_Supremo_789_12773 (tipo: Licenza)
Esecuzione query: MERGE (e:Licenza {nome: $nome}) SET e.ConsiglioIntergalatticoNumero = $ConsiglioIntergalatticoNumero, e.GiornoCicloCosmico = $GiornoCicloCosmico, e.Regolamentazion

In [138]:
# Estrai entità e relazioni
chunks = chunk_text(testo_blog1, 3000, 400)
for chunk in chunks:
    risultati = estrai_relazioni(chunk, relazioni, llm, driver)
    carica_entita_e_relazioni_su_neo4j(risultati, relazioni, driver)

Estraendo entità di tipo 'Piatto'...
Estraendo entità di tipo 'Licenza'...
Estraendo entità di tipo 'Pianeta'...
Estraendo entità di tipo 'Ingrediente'...
Estraendo entità di tipo 'Tecnica'...
Estraendo entità di tipo 'Ristorante'...
Estraendo relazioni 'CONTIENE' tra 'Piatto' e 'Ingrediente'...
Estraendo relazioni 'PREPARATO' tra 'Piatto' e 'Tecnica'...
Estraendo relazioni 'OFFERTA' tra 'Ristorante' e 'Piatto'...
Estraendo relazioni 'SITUATO' tra 'Ristorante' e 'Pianeta'...
Estraendo relazioni 'HA' tra 'Ristorante' e 'Licenza'...
Connessione al database verificata con successo
Elaborazione per tipo: Piatto
Tipo di dato: list
Caricando 3 entità di tipo Piatto...
Creazione entità: Risveglio del Drago Celeste (tipo: Piatto)
Esecuzione query: MERGE (e:Piatto {nome: $nome}) SET e.ingredient = $ingredient, e.percentage = $percentage RETURN e
Entità creata e confermata: Risveglio del Drago Celeste di tipo Piatto
Creazione entità: Cosmic Harmony Infusion (tipo: Piatto)
Esecuzione query: MERGE

In [139]:
# Estrai entità e relazioni
chunks = chunk_text(testo_blog2, 3000, 400)
for chunk in chunks:
    risultati = estrai_relazioni(chunk, relazioni, llm, driver)
    carica_entita_e_relazioni_su_neo4j(risultati, relazioni, driver)

Estraendo entità di tipo 'Piatto'...
Estraendo entità di tipo 'Licenza'...
Estraendo entità di tipo 'Pianeta'...
Estraendo entità di tipo 'Ingrediente'...
Estraendo entità di tipo 'Tecnica'...
Estraendo entità di tipo 'Ristorante'...
Estraendo relazioni 'CONTIENE' tra 'Piatto' e 'Ingrediente'...
Estraendo relazioni 'PREPARATO' tra 'Piatto' e 'Tecnica'...
Estraendo relazioni 'OFFERTA' tra 'Ristorante' e 'Piatto'...
Estraendo relazioni 'SITUATO' tra 'Ristorante' e 'Pianeta'...
Estraendo relazioni 'HA' tra 'Ristorante' e 'Licenza'...
Connessione al database verificata con successo
Elaborazione per tipo: Piatto
Tipo di dato: list
Caricando 5 entità di tipo Piatto...
Creazione entità: Sinfonia Quantistica delle Stelle (tipo: Piatto)
Esecuzione query: MERGE (e:Piatto {nome: $nome}) SET e.SaleTemporale = $SaleTemporale, e.NettareDiSirena = $NettareDiSirena RETURN e
Entità creata e confermata: Sinfonia Quantistica delle Stelle di tipo Piatto
Creazione entità: Galassia di Sapore Quantico (tipo:

In [141]:
import os
from pathlib import Path

# Percorso della cartella contenente i file PDF dei menu
menu_folder = "../Hackapizza Dataset/Menu/"

# Ottieni la lista di tutti i file PDF nella cartella
pdf_files = [f for f in os.listdir(menu_folder) if f.endswith('.pdf')]

# Per ogni file PDF nella cartella
for pdf_file in pdf_files:
    try:
        # Costruisci il percorso completo del file
        pdf_path = os.path.join(menu_folder, pdf_file)
        
        print(f"Elaborazione del file: {pdf_file}")
        
        # Estrai il testo dal PDF
        testo_menu = estrai_testo_pdf(pdf_path)
        
        risultati = estrai_relazioni(testo_menu, relazioni, llm, driver)
        carica_entita_e_relazioni_su_neo4j(risultati, relazioni, driver)
        
        print(f"Elaborazione di {pdf_file} completata con successo")
        print("-" * 50)
        
    except Exception as e:
        print(f"Errore durante l'elaborazione di {pdf_file}: {str(e)}")
        # Continua con il prossimo file anche in caso di errore
        continue

Elaborazione del file: L Essenza di Asgard.pdf
Estraendo entità di tipo 'Piatto'...
Estraendo entità di tipo 'Licenza'...
Estraendo entità di tipo 'Pianeta'...
Estraendo entità di tipo 'Ingrediente'...
Estraendo entità di tipo 'Tecnica'...
Estraendo entità di tipo 'Ristorante'...
Estraendo relazioni 'CONTIENE' tra 'Piatto' e 'Ingrediente'...
Estraendo relazioni 'PREPARATO' tra 'Piatto' e 'Tecnica'...
Estraendo relazioni 'OFFERTA' tra 'Ristorante' e 'Piatto'...
Estraendo relazioni 'SITUATO' tra 'Ristorante' e 'Pianeta'...
Estraendo relazioni 'HA' tra 'Ristorante' e 'Licenza'...
Connessione al database verificata con successo
Elaborazione per tipo: Piatto
Tipo di dato: list
Caricando 9 entità di tipo Piatto...
Creazione entità: Quadrifonia Cosmica: Sinfonia di Sapori e Dimensioni (tipo: Piatto)
Esecuzione query: MERGE (e:Piatto {nome: $nome}) SET e.INGREDIENTI = $INGREDIENTI, e.TECNICHE = $TECNICHE RETURN e
Entità creata e confermata: Quadrifonia Cosmica: Sinfonia di Sapori e Dimensioni 